In [1]:
!pip install torch==1.10.0 torchvision==0.11.1 torchtext==0.11.0 torchaudio==0.10.0 --quiet
!pip install pytorch-lightning==1.5.10 --quiet
!pip install lightning-flash==0.7.1 --quiet
!pip install 'lightning-flash[audio,image, video, text]' --quiet
!pip install Pillow==9.0.0

     |██████████████████████████████▎ | 834.1 MB 1.2 MB/s eta 0:00:39tcmalloc: large alloc 1147494400 bytes == 0x3e56000 @  0x7f4a694c6615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |████████████████████████████████| 881.9 MB 19 kB/s 
     |████████████████████████████████| 23.3 MB 1.8 MB/s 
     |████████████████████████████████| 8.0 MB 28.9 MB/s 
     |████████████████████████████████| 2.9 MB 30.7 MB/s 
     |████████████████████████████████| 527 kB 5.3 MB/s 
     |████████████████████████████████| 596 kB 39.8 MB/s 
     |████████████████████████████████| 829 kB 35.3 MB/s 
     |████████████████████████████████| 409 kB 40.5 MB/s 
     |████████████████████████████████| 952 kB 39.5 MB/s 
     |████████████████████████████████| 136 kB 45.3 MB/s 
     |██████

In [2]:
import pytorch_lightning as pl
import torch
import pandas as pd
import numpy as np
import seaborn as sns 

import flash
from flash.core.data.utils import download_data
from flash.video import VideoClassificationData, VideoClassifier

In [3]:
print("pandas version:",pd.__version__)
print("numpy version:",np.__version__)
print("seaborn version:",sns.__version__)
print("torch version:",torch.__version__)
print("flash version:",flash.__version__)
print("pytorch ligthening version:",pl.__version__)

pandas version: 1.3.5
numpy version: 1.21.6
seaborn version: 0.11.2
torch version: 1.10.0+cu102
flash version: 0.7.1
pytorch ligthening version: 1.5.10


In [4]:
import urllib3
urllib3.disable_warnings()

In [ ]:


download_data("https://pl-flash-data.s3.amazonaws.com/kinetics.zip", "./data")



./data/kinetics.zip:   0%|          | 0/165975 [00:00<?, ?KB/s]

In [ ]:
kinetics_videodatamodule = VideoClassificationData.from_folders(
    train_folder="data/kinetics/train",
    val_folder="data/kinetics/val",
    clip_sampler="uniform",
    clip_duration=2,
    decode_audio=False,
    batch_size=4
)

In [ ]:
# 4. List the available models
print(VideoClassifier.available_backbones())



['c2d_r50', 'csn_r101', 'efficient_x3d_s', 'efficient_x3d_xs', 'i3d_r50', 'r2plus1d_r50', 'slow_r50', 'slow_r50_detection', 'slowfast_16x8_r101_50_50', 'slowfast_r101', 'slowfast_r50', 'slowfast_r50_detection', 'x3d_l', 'x3d_m', 'x3d_s', 'x3d_xs']


In [ ]:

print(VideoClassifier.get_backbone_details("slow_r50"))

[('pretrained', <Parameter "pretrained: bool = False">), ('progress', <Parameter "progress: bool = True">), ('kwargs', <Parameter "**kwargs: Any">)]


In [ ]:

slowr50_model = VideoClassifier(backbone="slow_r50", labels=kinetics_videodatamodule.labels, pretrained=True)

Using 'slow_r50' provided by Facebook Research/PyTorchVideo (https://github.com/facebookresearch/pytorchvideo).
Downloading: "https://dl.fbaipublicfiles.com/pytorchvideo/model_zoo/kinetics/SLOW_8x8_R50.pyth" to /root/.cache/torch/hub/checkpoints/SLOW_8x8_R50.pyth


  0%|          | 0.00/248M [00:00<?, ?B/s]

In [ ]:
trainer = flash.Trainer(max_epochs=25, gpus=torch.cuda.device_count(), precision=16)
trainer.finetune(slowr50_model, datamodule=kinetics_videodatamodule, strategy="freeze")

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type       | Params
---------------------------------------------
0 | train_metrics | ModuleDict | 0     
1 | val_metrics   | ModuleDict | 0     
2 | test_metrics  | ModuleDict | 0     
3 | backbone      | Net        | 32.5 M
4 | head          | Sequential | 2.0 K 
---------------------------------------------
55.1 K    Trainable params
32.4 M    Non-trainable params
32.5 M    Total params
64.912    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/data.py:60: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 4. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  "Trying to infer the `batch_size` from an ambiguous collection. The batch size we"


Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/data.py:60: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 3. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  "Trying to infer the `batch_size` from an ambiguous collection. The batch size we"


...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:

trainer.save_checkpoint("finetuned_kinetics_slowr50_video_classification.pt")

In [ ]:

datamodule = VideoClassificationData.from_folders(predict_folder="data/kinetics/predict", batch_size=1)
predictions = trainer.predict(slowr50_model, datamodule=datamodule, output="labels")
print(predictions)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 58it [00:00, ?it/s]

[['archery'], ['marching'], ['bowling'], ['flying_kite'], ['high_jump']]
